# SQL
## Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Задания
Посчитайте, сколько книг вышло после 1 января 2000 года;
Для каждой книги посчитайте количество обзоров и среднюю оценку;
Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
Как выполнить задание?

## Ознакомимся с данными

In [2]:
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], db_config['pwd'], 
                                                         db_config['host'], db_config['port'], db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
query = '''

SELECT *
FROM books

LIMIT 5
'''

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''

SELECT *
FROM authors
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''

SELECT *
FROM publishers
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = '''

SELECT *
FROM ratings
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = '''

SELECT *
FROM reviews
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задачи

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [10]:
query = '''

SELECT COUNT(DISTINCT book_id)
FROM books
WHERE publication_date >= '2000-01-01'

'''

pd.io.sql.read_sql(query, con = engine)

,count
0,821


Всего с 1 января 2000 года вышло 821 книга. 

###  Для каждой книги посчитайте количество обзоров и среднюю оценку

In [13]:
query = '''

SELECT b.book_id, 
       b.title,
       COUNT(DISTINCT rev.review_id) AS cnt_reviews,
       ROUND(AVG(rating),2) AS avg_rating
FROM books AS b
LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
JOIN ratings AS rat ON b.book_id = rat.book_id

GROUP BY b.book_id, b.title
ORDER BY cnt_reviews DESC
'''

pd.io.sql.read_sql(query, con = engine)

,book_id,title,cnt_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


База насчитывает 1000 книг, 994 книги имеют хотя бы 1 обзор.

###  Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [14]:
query = '''

SELECT p.publisher,
       b.publisher_id 
       
FROM books AS b 
JOIN publishers AS p ON p.publisher_id = b.publisher_id

WHERE b.num_pages > 50

GROUP BY b.publisher_id, p.publisher
ORDER BY COUNT(b.book_id) DESC

LIMIT 1

'''

pd.io.sql.read_sql(query, con = engine)

,publisher,publisher_id
0,Penguin Books,212


Penguin Books выпустили наибольшее количество книг -212

###  Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [15]:
query = '''

WITH top AS
    (SELECT book_id, 
            COUNT(rating) As count_raiting
    FROM ratings

    GROUP BY book_id
    HAVING COUNT(rating) >=50)
    
SELECT a.author, AVG(r.rating) AS raiting
FROM books b
JOIN ratings AS r ON b.book_id = r.book_id
JOIN authors AS a ON b.author_id = a.author_id
JOIN top AS t ON b.book_id = t.book_id

GROUP BY a.author
ORDER BY raiting DESC

LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine)

,author,raiting
0,J.K. Rowling/Mary GrandPré,4.287097


Самая высокая средняя оценка у книг Дж. Роулинг

###  Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [16]:
query = '''
    
SELECT ROUND(AVG(review_cnt),2)
FROM (SELECT username, 
              COUNT(review_id) AS review_cnt
       FROM reviews 
       WHERE username IN (SELECT username
                          FROM ratings
                          GROUP BY username
                          HAVING COUNT(rating_id) > 50)
       GROUP BY  username) AS rereview_count

'''
pd.io.sql.read_sql(query, con = engine)

,round
0,24.33


Активные пользователи в среднем написали 24 обзора

## Вывод

- В базе содержатся 1000 книг, 994 книги имеют хотя бы 1 обзор. 
- С 1 января 2000 года вышла  821 новая книга.
- Издание Penguin Books выпустило наобильшее колличество книг, 212 новых книг.
- Самая высокая средняя оценка у книг Дж. Роулинг.
- Активные пользователи в среднем написали 24 обзора.